# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
print(plt.style.available)
plt.style.use('seaborn-deep')
DATA_FOLDER = '../../ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
data = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic')

In [ ]:
data.pclass.unique()

In [ ]:
data.survived.unique()

In [ ]:
data.sex.unique()

In [ ]:
data.embarked.unique()

In [ ]:
data.boat.unique()

In [ ]:
data.cabin.isnull().sum()

In [ ]:
len(data[data.body.notnull()].body.unique()) == len(data[data.body.notnull()].body)

In [ ]:
categories = {
    'pclass': 'category',
    #'survived': 'category',
    'sex': 'category',
    'embarked': 'category',
}

cities = {
    "C":"Cherbourg",
    "Q":"Queenstown",
    "S":"Southampton"
}

passengers = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic', dtype=categories)

passengers.embarked.cat.categories = [cities[c] for c in passengers.embarked.cat.categories]
passengers.embarked.cat.add_categories(["Unknown"], inplace=True)
passengers.embarked.fillna("Unknown", inplace=True)

passengers.survived = passengers.survived.astype('category', categories=[0, 1], ordered=True)

# Should inlcude unknown ages ? 
passengers["age_cat"] = pd.cut(passengers.age, range(0, np.ceil(passengers.age.max()).astype('int')+1, 10))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(15,15))
sns.countplot(x="pclass", data=passengers, ax=ax1);
sns.countplot(x='embarked', data=passengers, ax=ax2);
sns.countplot(x='sex', data=passengers, ax=ax3);
sns.countplot(x='age_cat', data=passengers, ax=ax4);

In [ ]:
def get_floor(o):
    if type(o) != str:
        return np.NaN
    else:
        floor_codes = list(set([c for c in o if c.isalpha()]))
        return floor_codes[0] if len(floor_codes) == 1 else np.NaN

passengers['floor'] = passengers.cabin.transform(get_floor).astype('category')

fig, axs = plt.subplots(figsize=(5,5))
passengers.floor.value_counts(sort=False).plot.pie(); #  We could have better precision by infering  missing value from class

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(15,5))
colors = ['lightcoral','darkseagreen']
labels = ['Died', 'Survived']
for (g, s), ax in zip(passengers.groupby(['pclass']).survived, axs):
    s.value_counts(sort=False).plot.pie(ax=ax, colors=colors, labels=labels)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
width = 0.25
for i, (grp_lab, grp) in enumerate(passengers.groupby(["pclass", "sex"]).survived):
    shift = (grp_lab[1]=='female')*(width+0.01)
    down = plt.bar(grp_lab[0]+shift, grp.value_counts()[0], width, color=colors[0])
    up = plt.bar(grp_lab[0]+shift, grp.value_counts()[1], width, bottom=grp.value_counts()[0], color=colors[1])

ax.set_xticks(passengers.pclass.cat.categories+0.5*width)
ax.set_xticklabels(passengers.pclass.cat.categories)

ax.set_title('Surviva count per class (Male, Female)')
ax.set_ylabel('Count')
ax.set_xlabel('Class (Male, Female)')
plt.legend(['Died', 'Survived'], loc='upper left');

Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. Present your results in a DataFrame with unique index.

In [ ]:
print(passengers.age.isnull().sum())
print(passengers.age.mean())
pd.qcut(passengers.age, [0, 0.5, 1]).value_counts()